## Yolov5 high resolution training

### Major modification
* img=3600
* mixup=0.5
* fliplr: 0.5

### Hardware to reproduce
* RTX3090

### Training Log:
> ```
version=1
img_size:3584,bs2,e11,[yolov5s6] 
Fold: video2[validation]
Labels: only GT
```

In [1]:
!nvidia-smi

Thu Feb 10 19:47:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

In [3]:
# train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
# train['pos'] = train.annotations != '[]'

In [4]:
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

# Turn annotations from strings into lists of dictionaries
df['annotations1'] = df['annotations'].apply(eval)

# Create the image path for the row
df['image_path'] = "video_" + df['video_id'].astype(str) + "/" + df['video_frame'].astype(str) + ".jpg"

length = lambda x: len(x) 

df["no_of_bbox"] = df["annotations1"].apply(length)

df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,annotations1,image_path,no_of_bbox
0,0,40258,0,0,0-0,[],[],video_0/0.jpg,0
1,0,40258,1,1,0-1,[],[],video_0/1.jpg,0
2,0,40258,2,2,0-2,[],[],video_0/2.jpg,0
3,0,40258,3,3,0-3,[],[],video_0/3.jpg,0
4,0,40258,4,4,0-4,[],[],video_0/4.jpg,0


In [5]:
val_df = df[df["video_id"]==0][df["no_of_bbox"]>0]
val_df.shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(2143, 9)

In [6]:
train_df = df[df["video_id"]!=0][df["no_of_bbox"]>0]
train_df.shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(2776, 9)

In [7]:
!mkdir -p ./yolo_data/fold0/images/val
!mkdir -p ./yolo_data/fold0/images/train

!mkdir -p ./yolo_data/fold0/labels/val
!mkdir -p ./yolo_data/fold0/labels/train

In [8]:
fold = 0

annos = []
for i, x in val_df.iterrows():
#     if x.video_id == fold:
#         if x.pos:
    mode = 'val'
#     else:
#         # train
#         mode = 'train'
#         if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
#     if not x.pos:
#         continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [9]:
fold = 0

annos = []
for i, x in train_df.iterrows():
#     if x.video_id == fold:
#         if x.pos:
    mode = 'train'
#     else:
#         # train
#         mode = 'train'
#         if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
#     if not x.pos:
#         continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

In [10]:
# fold = 1

# annos = []
# for i, x in train.iterrows():
#     if x.video_id == fold:
#         if x.pos:
#             mode = 'val'
#     else:
#         # train
#         mode = 'train'
#         if not x.pos: continue
#         # val
#     copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
#                 f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
#     if not x.pos:
#         continue
#     r = ''
#     anno = eval(x.annotations)
#     for an in anno:
# #            annos.append(an)
#         r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
#                                         (an['y'] + an['height'] / 2) / 720,
#                                         an['width'] / 1280, an['height'] / 720)
#     with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
#         fp.write(r)

In [11]:
import os
len(os.listdir("./yolo_data/fold0/labels/train"))

2776

In [12]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

In [13]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]

path: ../yolo_data/fold0/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [14]:
# !git clone https://github.com/ultralytics/yolov5.git
!git clone https://github.com/soumya997/yolov5-w-f2-mod.git
!mv ./yolov5-w-f2-mod ./yolov5

Cloning into 'yolov5-w-f2-mod'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 132 (delta 36), reused 120 (delta 26), pack-reused 0
Receiving objects: 100% (132/132), 1.39 MiB | 4.79 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [15]:
with open('./yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [16]:
%cd yolov5

/kaggle/working/yolov5


In [17]:
!ls data/

Argoverse.yaml	      VOC.yaml	     hyps		 xView.yaml
GlobalWheat2020.yaml  VisDrone.yaml  images
Objects365.yaml       coco.yaml      reef_f1_naive.yaml
SKU-110K.yaml	      coco128.yaml   scripts


In [18]:
!python -m wandb disabled

!python train.py \
    --img 3584 \
    --batch 2 \
    --epochs 11 \
    --data data/reef_f1_naive.yaml \
    --weights yolov5s6.pt \
    --name base_vid_2val \
    --hyp data/hyps/hyp.heavy.2.yaml \
    --save-period 1

W&B disabled.
100%|██████████████████████████████████████| 24.5M/24.5M [00:00<00:00, 51.1MB/s]

train: Scanning '../yolo_data/fold0/labels/train' images and labels...2776 found
val: Scanning '../yolo_data/fold0/labels/val' images and labels...2143 found, 0 
      0/10     13.2G   0.07685    0.1247         0         5      3584: 100%|███
               Class     Images     Labels          P          R         F2     
      1/10     13.2G   0.05546   0.09834         0        34      3584: 100%|███
               Class     Images     Labels          P          R         F2     
      2/10     13.2G    0.0507   0.08906         0        28      3584: 100%|███
               Class     Images     Labels          P          R         F2     
      3/10     13.2G   0.04476   0.08653         0        18      3584: 100%|███
               Class     Images     Labels          P          R         F2     
      4/10     13.2G   0.04137   0.08215         0         5      3584: 100%|███
             

In [19]:
# !python -m wandb disabled

# !python train.py \
#     --img 3000 \
#     --batch 2 \
#     --epochs 11 \
#     --data data/reef_f1_naive.yaml \
#     --weights yolov5s6.pt \
#     --name cots_with_albs \
#     --hyp data/hyps/hyp.heavy.2.yaml \
#     --save-period 1

In [20]:
!ls

CONTRIBUTING.md  __pycache__  hubconf.py	setup.cfg	val.py
Dockerfile	 data	      models		train.py	wandb
LICENSE		 detect.py    requirements.txt	tutorial.ipynb	yolov5s6.pt
README.md	 export.py    runs		utils


In [21]:
%cd /kaggle/working

!cp -r /kaggle/working/yolov5/runs/train/base_vid_2val /kaggle/working

!cp /kaggle/working/yolov5/data/reef_f1_naive.yaml /kaggle/working/base_vid_2val/
!cp /kaggle/working/yolov5/data/hyps/hyp.heavy.2.yaml /kaggle/working/base_vid_2val/
# !cp /kaggle/working/yolov5/utils/augmentations.py /kaggle/working/base_vid_2val/

!rm -r /kaggle/working/yolov5
!rm -r /kaggle/working/images
!rm -r /kaggle/working/labels

/kaggle/working
rm: cannot remove '/kaggle/working/images': No such file or directory
rm: cannot remove '/kaggle/working/labels': No such file or directory
